In [0]:
%run "./(Clone) wet_files_extraction"

In [0]:
pip install thefuzz

In [0]:
import re
from pyspark.sql.functions import lower
from pyspark.ml import Pipeline
from thefuzz import fuzz, process
from typing import Iterator
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, NGram, CountVectorizer, IDF, VectorAssembler, StringIndexer, HashingTF
from pyspark.ml.classification import LogisticRegression, LinearSVC, RandomForestClassifier, OneVsRest
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import mlflow, mlflow.spark

In [0]:
#runnning time ~2 mins
df = warc_en.select('target_uri','body','host','base_domain','lang')
display(df.limit(50))
print('English docs:', df.count())

#### Fuzzy Matching

In [0]:
# Setting body of text into lower case

df = (
    df.withColumn('body', F.lower(F.col('body')))
)

# Dictionary for Census and ACS general terminology for Fuzzy Matching 

GATES = {
    # Mention ACS acronym or any two words from the phrase (more forgiving)
    'ACS (American Community Survey)': re.compile(
        r'\b(acs|american\s+community|community\s+survey|american\s+survey)\b', re.I
    ),

    # Allow "census" alone OR "census ... bureau" (some pages omit "bureau")
    'U.S. Census Bureau': re.compile(
        r'(\bcensus\b.*\b(bureau|bea(?:u|ue|eau)r(?:o|ou|eau)|buro)\b|\bcensus\b)', re.I
    ),

    # Catch variants like "1 yr", "one-year", "1-year", "1 year"
    'ACS (1-year)': re.compile(
        r'\bacs\b.*\b(1|one)\s*[- ]?\s*year\b', re.I
    ),

    'ACS (5-year)': re.compile(
        r'\bacs\b.*\b(5|five)\s*[- ]?\s*year\b', re.I
    ),
}

# Targets dictionary
targets = {
    'american community survey': 'ACS (American Community Survey)',
    'u.s. census bureau':        'U.S. Census Bureau',
    'acs 1-year':                'ACS (1-year)',
    'acs 5-year':                'ACS (5-year)',
}

# Anchors (broader, to ensure we keep candidate coverage)
anchors = r'(american|community|survey|\bacs\b|census|bureau|bea(?:u|ue|eau)r(?:o|ou|eau)|buro)'



In [0]:
# normilizing text

def normalize(col='body'):
    return F.trim(F.regexp_replace(F.lower(F.col(col)), r'\s+', ' '))

docs = (
    df.withColumn('text_norm', normalize('body'))
      .filter(F.length('text_norm') > 50)
)


In [0]:
# Tokenizer
tokenizer = RegexTokenizer(
    inputCol='text_norm',
    outputCol='tokens',
    pattern=r'[^a-z0-9]+',
    gaps=True
)
tok_raw = tokenizer.transform(docs)  # docs is your filtered/normalized DF

# 2) Add NGram columns (this is what creates ng1..ng4)
for n in [1, 2, 3, 4]:
    tok_raw = NGram(n=n, inputCol='tokens', outputCol=f'ng{n}').transform(tok_raw)

empty_arr = F.lit([]).cast(T.ArrayType(T.StringType()))
for n in [1, 2, 3, 4]:
    if f'ng{n}' not in tok_raw.columns:
        # If for any reason NGram didn’t create a column, create an empty one
        tok_raw = tok_raw.withColumn(f'ng{n}', empty_arr)
    else:
        tok_raw = tok_raw.withColumn(f'ng{n}', F.coalesce(F.col(f'ng{n}'), empty_arr))



In [0]:
# Run time approximately ~8 mins

# Merge & explode all n-grams into candidate phrases
tok_exploded = (
    tok_raw
    .withColumn('all_ngrams', F.array_union(F.col('ng1'), F.col('ng2')))
    .withColumn('all_ngrams', F.array_union(F.col('all_ngrams'), F.col('ng3')))
    .withColumn('all_ngrams', F.array_union(F.col('all_ngrams'), F.col('ng4')))
    .withColumn('cand', F.explode('all_ngrams'))
    .select('target_uri', 'base_domain', 'text_norm', 'cand')
)

print('tok_exploded.count() =', tok_exploded.count())
print('distinct pages in tok_exploded =', tok_exploded.select('target_uri').distinct().count())



In [0]:
tok = tok_exploded.filter(F.col('cand').rlike(anchors))

# Reduce per-page explosion (keep up to 3k distinct candidates/page)
tok = (tok
    .withColumn('tok_cnt', F.size(F.split(F.col('cand'), r'\s+')))
    .filter(F.col('tok_cnt') >= 2)
    .filter(F.length('cand') >= 5)
    .drop('tok_cnt')
)

# Ensure target_uri exists and is non-null before window
if 'target_uri' not in tok.columns:
    tok = tok.withColumn('target_uri', F.concat_ws('-', F.lit('row'), F.monotonically_increasing_id()))
else:
    tok = tok.filter(F.col('target_uri').isNotNull())


w = Window.partitionBy('target_uri').orderBy(F.length('cand'))
tok = (
    tok.withColumn('rn', F.row_number().over(w))
       .filter(F.col('rn') <= 3000)
       .drop('rn')
)

# Collect candidates per page
cand_by_page = (
    tok.groupBy('target_uri','base_domain')
       .agg(F.collect_set('cand').alias('cand_list'))
       .join(docs.select('target_uri','text_norm'), on='target_uri', how='left')
)

cand_by_page = cand_by_page.withColumn(
    'cand_list',
    F.coalesce(
        F.col('cand_list'),
        F.lit([]).cast(T.ArrayType(T.StringType()))
    )
)

# Cleaning it up: trim, drop null/empty, de-dupe (optional but helpful)
cand_by_page = cand_by_page.withColumn(
    'cand_list',
    F.array_distinct(
        F.expr("filter(transform(cand_list, x -> trim(x)), x -> x is not null and x <> '')")
    )
)

In [0]:
# Disable all gates for one run
GATES = {v: None for v in targets.values()}
# re-run score_page_batch + scored

In [0]:
schema = T.StructType([
    T.StructField('target_uri',   T.StringType(), False),
    T.StructField('base_domain',  T.StringType(), True),
    T.StructField('target_label', T.StringType(), False),
    T.StructField('target_text',  T.StringType(), False),
    T.StructField('best_phrase',  T.StringType(), True),
    T.StructField('best_score',   T.IntegerType(), True),
])

targets_pdf = pd.DataFrame([(k, v) for k, v in targets.items()],
                           columns=['target_text', 'target_label'])

def score_page_batch(rows: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for df_batch in rows:
        out_rows = []
        for _, r in df_batch.iterrows():
            cands = r['cand_list'] if isinstance(r.get('cand_list', None), list) else []
            cands = [c.strip() for c in cands if isinstance(c, str) and c.strip()]

            for _, t in targets_pdf.iterrows():
                label = t['target_label']
                gate = GATES.get(label)

                # Gate
                gated_cands = [c for c in cands if gate.search(c)] if (gate and cands) else cands

                # 🔁 Fallback: if gate filtered away everything, use all candidates
                if not gated_cands and cands:
                    gated_cands = cands

                if gated_cands:
                    # lenient to find signal; switch to token_sort_ratio later if needed
                    match = process.extractOne(t['target_text'], gated_cands, scorer=fuzz.token_set_ratio)
                    best_phrase, best_score = (match[0], int(match[1])) if match else (None, None)
                else:
                    best_phrase, best_score = None, None

                out_rows.append({
                    'target_uri':   r['target_uri'],
                    'base_domain':  r.get('base_domain'),
                    'target_label': label,
                    'target_text':  t['target_text'],
                    'best_phrase':  best_phrase,
                    'best_score':   best_score,
                })
        yield pd.DataFrame(out_rows)

scored = cand_by_page.mapInPandas(score_page_batch, schema)

In [0]:
print('rows with non-null best_score:',
      scored.filter(F.col('best_score').isNotNull()).count())

display(
    scored.orderBy(F.desc('best_score'))
          .select('target_label','best_score','best_phrase','base_domain','target_uri')
          .limit(30)
)

In [0]:
# running time approximately ~8 mins

thresh = 80
scored_flagged = scored.withColumn('is_match', F.col('best_score') >= F.lit(thresh))

print('Total scored rows:', scored.count())
print('Matches above threshold:', scored_flagged.filter('is_match').count())




In [0]:
display(
    scored
      .select('target_label','best_score','best_phrase','base_domain','target_uri')
      .where(F.col('best_score').isNotNull())
      .orderBy(F.desc('best_score'))
      .limit(50)
)

In [0]:
print("tok columns:", tok.columns)
tok.select('target_uri').show(5)

In [0]:
# See the top matches to sanity-check quality
display(
    scored_flagged
      .filter('is_match')
      .select('target_label','best_score','best_phrase','base_domain','target_uri')
      .orderBy(F.desc('best_score'))
      .limit(50)
)

# Score distribution to decide final threshold
score_dist = (
    scored.filter(F.col('best_score').isNotNull())
          .groupBy('target_label')
          .agg(
              F.expr('percentile_approx(best_score, 0.5)').alias('p50'),
              F.expr('percentile_approx(best_score, 0.9)').alias('p90'),
              F.max('best_score').alias('max_score'),
              F.count('*').alias('pages_scored')
          )
)
display(score_dist)

In [0]:
# Any target matched on each page?
page_any = (
    scored_flagged.groupBy('target_uri')
    .agg(
        F.max(F.col('is_match').cast('int')).alias('any_match')
    )
)

# Sanity check counts
print('Total scored rows:', scored.count())
print('Matches above threshold:', scored_flagged.filter('is_match').count())

In [0]:
# Count distinct websites with ACS/Census matches
domain_counts = (
    scored_flagged
    .filter(F.col('is_match') == True)
    .select('base_domain')
    .distinct()
    .groupBy()
    .agg(
        F.count('*').alias('num_websites_with_terminology')
    )
)
display(domain_counts)

# Per-target website counts
per_target_domains = (
    scored_flagged
    .filter((F.col('is_match') == True) & F.col('base_domain').isNotNull())
    .groupBy('target_label')
    .agg(F.approx_count_distinct('base_domain', 0.02).alias('distinct_websites'))
    .orderBy(F.desc('distinct_websites'))
)
display(per_target_domains)

# Score distribution for threshold tuning
score_dist = (
    scored_flagged
    .filter(F.col('best_score').isNotNull())
    .groupBy('target_label')
    .agg(
        F.expr('percentile_approx(best_score, 0.5)').alias('p50'),
        F.expr('percentile_approx(best_score, 0.9)').alias('p90'),
        F.max('best_score').alias('max_score'),
        F.count('*').alias('pages_scored')
    )
    .orderBy('target_label')
)

display(score_dist)

In [0]:
print('docs rows:', docs.count())
print('candidate rows (tok):', tok.count())
print('pages with candidates:', tok.select('target_uri').distinct().count())
print('pages after scoring:', scored.select('target_uri').distinct().count())

display(
    scored.orderBy(F.desc('best_score'))
           .select('target_label','best_score','best_phrase','base_domain','target_uri')
           .limit(20)
)

In [0]:
# Sampling for manual inspection
top_hits = (
    scored_flagged
    .filter(F.col('is_match'))
    .select('target_label','best_score','best_phrase','base_domain','target_uri')
    .orderBy(F.desc('best_score'))
    .limit(100)
)
display(top_hits)

****

In [0]:
## --- 1) Lowercase + create text_norm (you can add more cleaning later if you want) ---
#df = (
#    df.withColumn("body", F.lower(F.col("body")))
#      .withColumn("text_norm", F.col("body"))   # ensure it exists for regex below
#)
#
## --- 2) Regex features + weak labels ---
#acs_phrase_rx    = r"(american community survey|acs(?![a-z0-9]))"
#census_phrase_rx = r"(u\.?s\.?\s*census\s*bureau|united states census bureau|us census bureau)"
#acs_year_rx      = r"(acs\s*1[- ]?year|acs\s*5[- ]?year|acs\s+\d{4})"
#table_id_rx      = r"\b(?:b|c|s|dp)\d{3,6}\b"      # B01003, C17002, S1901, DP03...
#source_line_rx   = r"(source\s*:\s*(?:u\.?s\.?\s*)?census\s*bureau.*acs)"
#
#df = (
#    df
#    .withColumn("has_acs_phrase",    F.col("text_norm").rlike(acs_phrase_rx))
#    .withColumn("has_census_bureau", F.col("text_norm").rlike(census_phrase_rx))
#    .withColumn("has_acs_year",      F.col("text_norm").rlike(acs_year_rx))
#    .withColumn("has_table_id",      F.col("text_norm").rlike(table_id_rx))
#    .withColumn("has_source_line",   F.col("text_norm").rlike(source_line_rx))
#    .withColumn(
#        "weak_label",
#        F.when(F.col("has_source_line") | (F.col("has_census_bureau") & F.col("has_acs_phrase")), 1)
#         .when(F.col("has_table_id") & ~F.col("has_census_bureau") & ~F.col("has_source_line"), 0)
#         .otherwise(F.lit(None))
#    )
#)

In [0]:
## --- 3) Fuzzy score (optional; only computed on likely ACS pages) ---
#@F.udf(T.IntegerType())
#def acs_fuzzy_score(t):
#    if t is None:
#        return None
#    t = t[:5000]
#    return int(max(fuzz.partial_ratio(t, pat) for pat in [
#        "source: u.s. census bureau, american community survey",
#        "american community survey (acs) 5-year estimates",
#        "u.s. census bureau acs table",
#    ]))
#
#df = df.withColumn(
#    "acs_fuzzy",
#    F.when(
#        F.col("has_acs_phrase") | F.col("has_census_bureau") | F.col("has_table_id"),
#        acs_fuzzy_score(F.col("text_norm"))
#    )
#).limit(10)
#

In [0]:
## Running time approximately `29 sec
#
## --- 4) Build ML set safely (no empty docs/vocab) ---
#data = (
#    df.filter(F.length("text_norm") > 100)
#      .withColumn("f_acs_fuzzy",        F.coalesce(F.col("acs_fuzzy").cast("double"), F.lit(0.0)))
#      .withColumn("f_has_acs_phrase",    F.col("has_acs_phrase").cast("double"))
#      .withColumn("f_has_census_bureau", F.col("has_census_bureau").cast("double"))
#      .withColumn("f_has_acs_year",      F.col("has_acs_year").cast("double"))
#      .withColumn("f_has_table_id",      F.col("has_table_id").cast("double"))
#      .withColumn("f_has_source_line",   F.col("has_source_line").cast("double"))
#)
#
#tok  = RegexTokenizer(inputCol="text_norm", outputCol="tok", pattern=r"\W+")
#stop = StopWordsRemover(inputCol="tok", outputCol="tok_sw")
#tmp  = Pipeline(stages=[tok, stop]).fit(data).transform(data)
#tmp  = tmp.filter(F.size("tok_sw") > 0)   # avoid empty-token rows
#
## Labeled set (fallback to guarantee at least some labels)
#labeled = tmp.filter(F.col("weak_label").isNotNull())
#if labeled.limit(1).count() == 0:
#    labeled = tmp.withColumn(
#        "label",
#        F.when(F.col("has_source_line") | (F.col("has_census_bureau") & F.col("has_acs_phrase")), 1).otherwise(0)
#    )
#else:
#    labeled = labeled.withColumnRenamed("weak_label", "label")
#
#ng2   = NGram(n=2, inputCol="tok_sw", outputCol="tok2")
#
# Text features (relaxed thresholds avoid empty vocabulary)
#htf1  = HashingTF(inputCol="tok_sw", outputCol="tf1", numFeatures=1<<16)  # 65,536
#htf2  = HashingTF(inputCol="tok2",   outputCol="tf2", numFeatures=1<<16)  # 65,536
#
#idf1  = IDF(inputCol="tf1", outputCol="tfidf1")
#idf2  = IDF(inputCol="tf2", outputCol="tfidf2")
#
#asm = VectorAssembler(
#    inputCols=["tfidf1","tfidf2","f_has_acs_phrase","f_has_census_bureau",
#               "f_has_acs_year","f_has_table_id","f_has_source_line","f_acs_fuzzy"],
#    outputCol="features"
#)

In [0]:
##This is taking approximately ~10 mins to run. It builds individual predictions by Website.
#
#lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=50)
#
#pipe = Pipeline(stages=[ng2, htf1, htf2, idf1, idf2, asm, lr])
#model = pipe.fit(labeled)
#pred  = model.transform(tmp).select("target_uri","prediction","probability")
#display(pred)